## R3 To R1 Point Attractor, S2 Damping, R1 Sphere Avoidance

In [1]:
using PBDS, StaticArrays, LinearAlgebra, BenchmarkTools

HTML{String}("<script>\n// Immediately-invoked-function-expression to avoid global variables.\n(function() {\n    var warning_div = document.getElementById(\"webio-warning-11931108310396936154\");\n    var hide = function () {\n        var script = document.getElementById(\"webio-setup-15252530834260230912\");\n        var parent = script && script.parentElement;\n        var grandparent = parent && parent.parentElement;\n        if (grandparent) {\n            grandparent.style.display = \"none\";\n        }\n        warning_div.style.display = \"none\";\n    };\n    if (typeof Jupyter !== \"undefined\") {\n        console.log(\"WebIO detected Jupyter notebook environment.\");\n        // Jupyter notebook.\n        var extensions = (\n            Jupyter\n            && Jupyter.notebook.config.data\n            && Jupyter.notebook.config.data.load_extensions\n        );\n        if (extensions && extensions[\"webio-jupyter-notebook\"]) {\n            // Extension already loaded.\n            console.log(\"Jupyter WebIO nbextension detected; not loading ad-hoc.\");\n            hide();\n            return;\n        }\n    } else if (window.location.pathname.includes(\"/lab\")) {\n        // Guessing JupyterLa\n        console.log(\"Jupyter Lab detected; make sure the @webio/jupyter-lab-provider labextension is installed.\");\n        hide();\n        return;\n    }\n})();\n\n</script>\n<p\n    id=\"webio-warning-11931108310396936154\"\n    class=\"output_text output_stderr\"\n    style=\"padding: 1em; font-weight: bold;\"\n>\n    Unable to load WebIO. Please make sure WebIO works for your Jupyter client.\n    For troubleshooting, please see <a href=\"https://juliagizmos.github.io/WebIO.jl/latest/providers/ijulia/\">\n    the WebIO/IJulia documentation</a>.\n    <!-- TODO: link to installation docs. -->\n</p>\n")

## Setup

### Point Distance Attractor

In [2]:
PBDS.metric_chart(xn, task::Attractor{<:DistanceFromPoint{R3,R1,S}}, CN::Chart{1,R1}) where S = 
    default_metric(xn, task, CN)
PBDS.potential_chart(xn, task::Attractor{<:DistanceFromPoint{R3,R1,S}}, CN::Chart{1,R1}) where S = xn[1]^2
PBDS.dissipative_forces_chart(xn, vn, task::Attractor{<:DistanceFromPoint{R3,R1,S}}, CN::Chart{1,R1}) where S =  -5*vn
PBDS.weight_metric_chart(xn, vn, task::Attractor{<:DistanceFromPoint{R3,R1,S}}, CN::Chart{1,R1}) where S = 
    default_weight_metric(xn, vn, task, CN)

### Angular Damping Around Goal

In [3]:
PBDS.default_coord_rep(::Damping{<:AngularPositionAroundPoint{R3,S2,S}}) where S = EmbRep()
PBDS.codomain_coord_rep(::AngularPositionAroundPoint{R3,S2,S}) where S = EmbRep()
PBDS.metric_emb(xne, task::Damping{<:AngularPositionAroundPoint{R3,S2,S}}) where S =
    default_metric(xne, task)
PBDS.potential_emb(xne, task::Damping{<:AngularPositionAroundPoint{R3,S2,S}}) where S = 0.
PBDS.dissipative_forces_emb(xne, vne, task::Damping{<:AngularPositionAroundPoint{R3,S2,S}}) where S = -2*vne
PBDS.weight_metric_emb(xne, vne, task::Damping{<:AngularPositionAroundPoint{R3,S2,S}}) where S =
    default_weight_metric(xne, vne, task)

### Sphere Avoidance

In [4]:
function PBDS.metric_chart(xn, task::CollisionAvoidance{<:DistanceFromSphereSurface{R3,R1,S}}, CN::Chart{1,R1}) where S
    ψx = exp(1.e1 / xn[1]^2)
    G = SMatrix{1,1,eltype(xn)}([ψx])
end
PBDS.potential_chart(xn, task::CollisionAvoidance{<:DistanceFromSphereSurface{R3,R1,S}}, CN::Chart{1,R1}) where S = 0.
PBDS.dissipative_forces_chart(xn, vn, task::CollisionAvoidance{<:DistanceFromSphereSurface{R3,R1,S}}, CN::Chart{1,R1}) where S = 0*vn
function PBDS.weight_metric_chart(xn, vn, task::CollisionAvoidance{<:DistanceFromSphereSurface{R3,R1,S}}, CN::Chart{1,R1}) where S
    offset_distance = 5.
    λ = (xn[1] < offset_distance && vn[1] < 0.) ? 1. : 0.
    W = SMatrix{1,1,eltype(xn)}(I)*λ
end

# <span style="color:red">Sid: adjust tasks below</span>.

In [5]:
xm_goal = SA[-6., 6., 0.]
# xm_goal = SA[-7., 7., 1.]
M = R3
CM = Chart{1,M}()
tasks, CNs = TaskList(), ChartList()

N = R1
push!(tasks, Attractor(DistanceFromPoint{M,N}(xm_goal)))
push!(CNs, Chart{1,N}())

N = S2
push!(tasks, Damping(AngularPositionAroundPoint{M,N}(xm_goal)))
push!(CNs, Chart{SterProjSouth,N}())

N = R1
CN = Chart{1,N}()
center = SA[-4., 4., 0.]
radius = 2.
push!(tasks, CollisionAvoidance(DistanceFromSphereSurface{M,N}(center, radius)))
push!(CNs, CN)

3-element Array{Chart,1}:
 Chart{1,ℝ{1}}()
 Chart{SterProjSouth,𝕊{2}}()
 Chart{1,ℝ{1}}()

## Point Acceleration

In [6]:
# Initial state
xm = SA[3., 4., 0.]
vm = SA[1., 1., 0.]
σxddot, = multiple_task_acceleration(xm, vm, tasks, CM, CNs)

"Look at single task component"

"Look inside the loop"

"Look at single task component"

"Look inside the loop"

"Look at single task component"

"Look inside the loop"

"Look at this"

3×3 SArray{Tuple{3,3},Float64,2,9} with indices SOneTo(3)×SOneTo(3):
  0.958858   -0.211765   0.0177515
 -0.211765    0.052976   0.0236686
  0.0177515   0.0236686  0.147929

([-32.49602286258665, -43.02033817575599, 10.37046915932385], Chart[])

## Single Trajectory Test

In [7]:
Time = 1
dt = 0.05

traj = propagate_tasks(xm, vm, tasks, CM, CNs, Time, dt)
traj.xm[end]

"Look at single task component"

"Look inside the loop"

"Look at single task component"

"Look inside the loop"

"Look at single task component"

...

"Look at single task component"

"Look inside the loop"

"Look at this"

3×3 SArray{Tuple{3,3},Float64,2,9} with indices SOneTo(3)×SOneTo(3):
  1.44384     0.79468    -0.0389439
  0.79468     0.556378   -0.0294109
 -0.0389439  -0.0294109   0.0599564

3-element SArray{Tuple{3},Float64,1,3} with indices SOneTo(3):
  0.49732218741971346
  8.137466406284645
 -0.14881671273612998

## Multiple Trajectories

In [18]:
using Makie, Observables, ProgressMeter, VectorizedRoutines

┌ Info: Precompiling ProgressMeter [92933f4c-e287-5a05-a399-4b506db050ca]
└ @ Base loading.jl:1278
┌ Info: Precompiling VectorizedRoutines [0e69188a-a5d4-5622-b4e4-a72373136fc5]
└ @ Base loading.jl:1278
┌ Warning: Module Distributions with build ID 25504510479186 is missing from the cache.
│ This may mean Distributions [31c24e10-a181-5473-b8eb-7969acd0382f] does not support precompilation but is imported by a module that does.
└ @ Base loading.jl:1017
┌ Info: Skipping precompilation since __precompile__(false). Importing VectorizedRoutines [0e69188a-a5d4-5622-b4e4-a72373136fc5].
└ @ Base loading.jl:1034


In [19]:
Time = 11
dt = 0.03
xlim, Δx = 1., 2.
vlim, Δv = 2., 4.

xm0 = Array{Any}(undef, 3)
vm0 = Array{Any}(undef, 3)

xm0[1], xm0[2], xm0[3] = Matlab.meshgrid(-xlim:Δx:xlim, -xlim:Δx:xlim, -xlim:Δx:xlim)
vm0[1], vm0[2], vm0[3] = Matlab.meshgrid(-vlim:Δv:vlim, -vlim:Δv:vlim, -vlim:Δv:vlim)

for a in (xm0, vm0), i in 1:3
    a[i] = reshape(a[i], length(a[i]))
end

nx = length(xm0[1])
nv = length(vm0[1])
ix, iv = Matlab.meshgrid(1:nx, 1:nv)
ix = reshape(ix, length(ix))
iv = reshape(iv, length(iv))

ntraj = nx*nv
trajs = Array{Any}(undef, ntraj)
xm0 = [xm0[1]'; xm0[2]'; xm0[3]']
vm0 = [vm0[1]'; vm0[2]'; vm0[3]']
p = Progress(ntraj)

Threads.@threads for i in 1:ntraj
    trajs[i] = propagate_tasks(SA[xm0[:,ix[i]]...], SA[vm0[:,iv[i]]...], tasks, CM, CNs, Time, dt)
    next!(p)
end

"Look at single task component"

"Look inside the loop"

"Look at single task component"

"Culprit single task?"

Damping{AngularPositionAroundPoint{ℝ{3},𝕊{2},Float64,3}}(AngularPositionAroundPoint{ℝ{3},𝕊{2},Float64,3}([-6.0, 6.0, 0.0]))

3×3 SArray{Tuple{3,3},Float64,2,9} with indices SOneTo(3)×SOneTo(3):
 NaN  NaN  NaN
 NaN  NaN  NaN
 NaN  NaN  NaN

2×3 SArray{Tuple{2,3},Float64,2,6} with indices SOneTo(2)×SOneTo(3):
 NaN  NaN  NaN
 NaN  NaN  NaN

LoadError: [91mTaskFailedException:[39m
[91m[91msingle task error checking[39m[39m
[91mStacktrace:[39m
[91m [1] [1merror[22m[1m([22m::String[1m)[22m at [1m./error.jl:33[22m[39m
[91m [2] [1msingle_task_components[22m[1m([22m::SArray{Tuple{3},Float64,1,3}, ::SArray{Tuple{3},Float64,1,3}, ::Damping{AngularPositionAroundPoint{ℝ{3},𝕊{2},Float64,3}}, ::Chart{1,ℝ{3}}, ::Chart{SterProjSouth,𝕊{2}}[1m)[22m at [1m/home/sid/.julia/dev/PBDS/src/pbds_tasks.jl:74[22m[39m
[91m [3] [1mmultiple_task_acceleration[22m[1m([22m::SArray{Tuple{3},Float64,1,3}, ::SArray{Tuple{3},Float64,1,3}, ::Array{PBDS.Task{var"#s13"} where var"#s13"<:Union{ProductTaskMap, PBDS.TaskMap},1}, ::Chart{1,ℝ{3}}, ::Array{Chart,1}, ::ChartRep; log_task_chart::Bool[1m)[22m at [1m/home/sid/.julia/dev/PBDS/src/pbds_tasks.jl:127[22m[39m
[91m [4] [1mPBDS.RMPTrajectory[22m[1m([22m::SArray{Tuple{3},Float64,1,3}, ::SArray{Tuple{3},Float64,1,3}, ::Int64, ::Float64, ::Array{PBDS.Task{var"#s13"} whe

In [20]:
# Sid: testing out variables to figure out Lapack error
xm0
vm0
tasks
CM
CNs
Time
dt

0.03

In [11]:
fxx(i) = [trajs[j].xm[i][1] for j = 1:ntraj]
fxy(i) = [trajs[j].xm[i][2] for j = 1:ntraj]
fxz(i) = [trajs[j].xm[i][3] for j = 1:ntraj]
iobs = Observable(1)

ax_size, plot_size = 6, 800
limits = FRect3D((-ax_size-3, -ax_size+3, -ax_size), (2*ax_size, 2*ax_size, 2*ax_size))
scene = Scene(resolution = (plot_size, plot_size))
Makie.scatter!(scene, lift(i -> fxx(i), iobs), lift(i -> fxy(i), iobs), lift(i -> fxz(i), iobs), markersize = ax_size/20, color = :blue, limits = limits)
Makie.scatter!(scene, xm_goal', markersize = ax_size/20, color = :darkgreen)
mesh!(Sphere(Point3f0(center), radius), color = :orange)
for i = 1:ntraj
    Makie.lines!(scene, getindex.(trajs[i].xm,1), getindex.(trajs[i].xm,2), getindex.(trajs[i].xm,3), color = :purple)
end

Makie.xlabel!(scene, "x")
Makie.ylabel!(scene, "y")
axis = scene[Axis]
axis.showaxis = false
display(scene)

LoadError: [91mUndefVarError: Observable not defined[39m

In [12]:
function record_scene(scene, filename, iobs, N, framerate=60)
    p = Progress(N)
    record(scene, filename, 1:N) do i
        iobs[] = i
        rotate_cam!(scene, 0.01, 0., 0.)
        next!(p)
    end
    display("text/html", html_video(filename))
end

filename = "R3_To_R1xS2PointDistanceAttractorXDamping_R1SphereAvoidance.mp4"
record_scene(scene, filename, iobs, length(trajs[1].xm))

LoadError: [91mUndefVarError: trajs not defined[39m

# Sid: testing things below here

In [13]:
? emb_to_chart

search: emb_to_chart



No documentation found.

`PBDS.emb_to_chart` is a `Function`.

```
# 10 methods for generic function "emb_to_chart":
[1] emb_to_chart(pe, ::Chart{SterProjSouth,𝕊{n}}) where n in PBDS at /home/sid/.julia/dev/PBDS/src/manifolds/SN.jl:20
[2] emb_to_chart(pe, ::Chart{SterProjNorth,𝕊{n}}) where n in PBDS at /home/sid/.julia/dev/PBDS/src/manifolds/SN.jl:21
[3] emb_to_chart(pe, ::Chart{Angleπ,𝕊{1}}) in PBDS at /home/sid/.julia/dev/PBDS/src/manifolds/SN.jl:46
[4] emb_to_chart(pe, C2π::Chart{Angle2π,𝕊{1}}) in PBDS at /home/sid/.julia/dev/PBDS/src/manifolds/SN.jl:47
[5] emb_to_chart(p, C::Chart{1,ℝ{n}}) where n in PBDS at /home/sid/.julia/dev/PBDS/src/manifolds/RN.jl:17
[6] emb_to_chart(p, C::Chart{2,ℝ{n}}) where n in PBDS at /home/sid/.julia/dev/PBDS/src/manifolds/RN.jl:21
[7] emb_to_chart(p, C::Chart{3,ℝ{n}}) where n in PBDS at /home/sid/.julia/dev/PBDS/src/manifolds/RN.jl:25
[8] emb_to_chart(p, C::Chart{4,ℝ{n}}) where n in PBDS at /home/sid/.julia/dev/PBDS/src/manifolds/RN.jl:28
[9] emb_to_chart(xe, C::Chart{I,M}) where {M<:ProductManifold, I} in PBDS at /home/sid/.julia/dev/PBDS/src/manifolds/product_manifolds.jl:29
[10] emb_to_chart(pe, C::Chart{I,TangentBundle{M}}) where {M, I} in PBDS at /home/sid/.julia/dev/PBDS/src/manifolds.jl:85
```


In [14]:
error("tsting")

LoadError: [91mtsting[39m

In [15]:
    # Sid: added a try-catch to determine the issue
    try
       error("testing error checking") 
    catch
        error("testing error checking") 
    end

LoadError: [91mtesting error checking[39m

In [16]:
display("testing")

"testing"

In [17]:
JftWJf_sum = zeros(3,3)
pinv(JftWJf_sum)

3×3 Array{Float64,2}:
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0